In [2]:
# -*- coding: utf-8 -*-

import netCDF4 as nc4
import numpy as np
import time, sys, os

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, FixedLocator, FixedFormatter,
                               NullLocator)
import matplotlib.gridspec as gridspec

import matplotlib.animation as animation
import types


In [1]:
dirt = "/gpfswork/rech/omr/ukd13rj/nemo/penalnemov4.0.1/tests/penaloverflow/exp_up1/"
#dirm = "/gpfswork/rech/bho/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/article_CORNER/MESHMASK/"

#workdir="min_phimin0001_smo6" ; savedir="up1"
workdir="tup1_uup1" ; savedir="up1"
#workdir="ref_zco_AimpT" ; savedir="up1"

pdt = dirt + workdir + "/OVF_grid_T.nc"
pdu = dirt + workdir + "/OVF_grid_U.nc"

save = 1 ; psave = "%s/%s" % (savedir,workdir)
dpi = 300  ; tskip = 4

In [3]:
dirt = "/gpfswork/rech/omr/ukd13rj/nemo/penalnemov4.0.1/tests/penaloverflow/exp_up1sigma/"
#dirm = "/gpfswork/rech/bho/ukd13rj/nemo/dev_r12527_Gurvan_ShallowWater/cfgs/article_CORNER/MESHMASK/"

workdir="important_min_phimin0001" ; savedir="up1"

pdt = dirt + workdir + "/OVF_grid_T.nc"
pdu = dirt + workdir + "/OVF_grid_U.nc"

save = 1 ; psave = "%s/%s" % (savedir,workdir)
dpi = 300  ; tskip = 4

In [4]:
dt = nc4.Dataset(pdt)
#
toce    = dt.variables['thetao_inst'][::tskip,:,1,:]
nT,nK,nI = np.shape(toce)
tmask = toce[0,:,:] < 1e-10
bigmask = np.repeat(tmask[np.newaxis,:,:],nT,axis=0)

toce = np.ma.masked_where(bigmask,toce)
cfltu   = np.ma.masked_where(bigmask, dt.variables['cfltu'][::tskip,:,1,:])
cfltw   = np.ma.masked_where(bigmask, dt.variables['cfltw'][::tskip,:,1,:])
lipst   = np.ma.masked_where(bigmask, dt.variables['lipst'][::tskip,:,1,:])
dt.close()

du = nc4.Dataset(pdu)
cfluu   = np.ma.masked_where(bigmask, du.variables['cfluu'  ][::tskip,:,1,:]) 
cfluw   = np.ma.masked_where(bigmask, du.variables['cfluw'  ][::tskip,:,1,:]) 
lipsu   = np.ma.masked_where(bigmask, du.variables['lipsu'  ][::tskip,:,1,:]) 
du.close()
print("data loaded")

data loaded


In [5]:
palette = plt.get_cmap('RdBu_r',10)
optpcolor = {"cmap" : palette}
optpcflt = {"cmap" : palette, "vmin" : np.min((np.min(cfltw),np.min(cfltu))), 
                              "vmax" : np.max((np.max(cfltw),np.max(cfltu)))}

optpcflu = {"cmap" : palette, "vmin" : np.min((np.min(cfluw),np.min(cfluu))), 
                              "vmax" : np.max((np.max(cfluw),np.max(cfluu)))}

optplips = {"cmap" : palette, "vmin" : np.min((np.min(lipst),np.min(lipsu))), 
                              "vmax" : np.max((np.max(lipst),np.max(lipsu)))}
optptoce = {"cmap" : palette, "vmin" : 10, "vmax" : 20}

NI = 2 ; NJ = 4 # nb of rows (NI) and cols (NJ)
fig, ax = plt.subplots(NI,NJ, figsize=(NJ*6,NI*4), dpi = dpi, squeeze=False)
# settled per rows (i=0) j=0 j=1 j=2
#                  (i=1) j=0 j=1 j=2

t=0
im = [[],[]]
"""Set the data for the ith iteration of the animation."""        
im[0].append( ax[0][0].imshow(toce [t]   , **optptoce)  ) ; ax[0][0].set_title('%s (%d/%d)' % (workdir,t,nT))
im[0].append( ax[0][1].imshow(cfltw[t]   , **optpcflt)  ) ; ax[0][1].set_title('cfltw max %.3f' % ( np.max(cfltw[t]) ) )
im[0].append( ax[0][2].imshow(cfluw[t]   , **optpcflu)  ) ; ax[0][2].set_title('cfluw max %.3f' % ( np.max(cfluw[t]) ) )
im[0].append( ax[0][3].imshow(lipst[t]   , **optplips)  ) ; ax[0][3].set_title('lipst max %.3f' % ( np.max(lipst[t]) ) )
""" 2nd row : x    cfltu cfluw lipsu
"""
im[1].append( ax[1][0].axis('off') )
im[1].append( ax[1][1].imshow(cfltu[t]   , **optpcflt) ) ; ax[1][1].set_title('cfltu max %.3f' % ( np.max(cfltu[t]) ) )
im[1].append( ax[1][2].imshow(cfluu[t]   , **optpcflu) ) ; ax[1][2].set_title('cfluw max %.3f' % ( np.max(cfluu[t]) ) )
im[1].append( ax[1][3].imshow(lipsu[t]   , **optplips) ) ; ax[1][3].set_title('lipsu max %.3f' % ( np.max(lipsu[t]) ) )

def animate(t):
    print("%d/%d" % (t,nT-1))
    """Set the data for the ith iteration of the animation."""      
    im[0][0].set_data(toce [t]) ; ax[0][0].set_title('%s (%d/%d)'     % ( workdir,t,nT     ) )
    im[0][1].set_data(cfltw[t]) ; ax[0][1].set_title('cfltw max %.3f' % ( np.max(cfltw[t]) ) )
    im[0][2].set_data(cfluw[t]) ; ax[0][2].set_title('cfluw max %.3f' % ( np.max(cfluw[t]) ) )
    im[0][3].set_data(lipst[t]) ; ax[0][3].set_title('lipst max %.3f' % ( np.max(lipst[t]) ) )

    """ 2nd row : x    cfltu cfluw lipsu
    """
    im[1][1].set_data(cfltu[t]) ; ax[1][1].set_title('cfltu max %.3f' % ( np.max(cfltu[t]) ) )
    im[1][2].set_data(cfluu[t]) ; ax[1][2].set_title('cfluw max %.3f' % ( np.max(cfluu[t]) ) )
    im[1][3].set_data(lipsu[t]) ; ax[1][3].set_title('lipsu max %.3f' % ( np.max(lipsu[t]) ) )
    #
    return ax

for i in range(NI):
        for j in range(NJ):
            ax[i][j].set_xticklabels([]) ; ax[i][j].set_yticklabels([])
            ax[i][j].set_xticks([])      ; ax[i][j].set_yticks([]) 
            ax[i][j].set_aspect(aspect='equal')
            ax[i][j].patch.set_color('0.8')
            if (i,j) != (1,0):
                fig.colorbar(im[i][j],ax=ax[i:(i+1),j:(j+1)], 
                             shrink=0.8, location='bottom',
                             pad = 0.02)
fig.patch.set_facecolor('white')

    
if save:
    anim = animation.FuncAnimation(fig, animate, frames=nT, blit=False, repeat=False)
    writer = animation.writers['ffmpeg'](fps=2)
    anim.save('%s.mp4' % (psave), writer=writer, dpi=dpi)
    plt.close("all")
    print("\nsaving : %s" % psave)
else:
    anim = animation.FuncAnimation(fig, animate, frames=nT)
    plt.show()

0/25
0/25
1/25
2/25
3/25
4/25
5/25
6/25
7/25
8/25
9/25
10/25
11/25
12/25
13/25
14/25
15/25
16/25
17/25
18/25
19/25
20/25
21/25
22/25
23/25
24/25
25/25

saving : up1/important_min_phimin0001


In [5]:
dirt = "/gpfswork/rech/omr/ukd13rj/nemo/penalnemov4.0.1/tests/penaloverflow/exp_up1/"
worklist = [
"min_phimin0001_smo0",
"min_phimin0001_smo1",
"min_phimin0001_smo2",
"min_phimin0001_smo3",
"min_phimin0001_smo4",
"min_phimin0001_smo5",
"min_phimin0001_smo6",
"min_phimin0001_smo7",
"min_phimin0001_smo8",
"min_phimin0001_smo9",
"min_phimin01_smo0",
"min_phimin01_smo1",
"min_phimin01_smo2",
"min_phimin01_smo3",
"min_phimin01_smo4",
"min_phimin01_smo5",
"moy_phimin0001_smo0",
"moy_phimin0001_smo1",
"moy_phimin0001_smo2",
"moy_phimin0001_smo3",
"moy_phimin0001_smo4"] # more was too much
#worklist = [
#"moy_phimin0001_smo5",
#"moy_phimin0001_smo6",
#"moy_phimin0001_smo7",
#"moy_phimin01_smo0",
#"moy_phimin01_smo1",
#"moy_phimin01_smo2",
#"moy_phimin01_smo3",
#"moy_phimin01_smo4",
#"moy_phimin01_smo5"]
nL = len(worklist)

In [7]:
dpi = 300 ; tskip = 4

palette = plt.get_cmap('RdBu_r',10)
optpcolor = {"cmap" : palette}
optptoce = {"cmap" : palette, "vmin" : 10, "vmax" : 20}
NI = 2 ; NJ = 4 # nb of rows (NI) and cols (NJ)

for l in range(nL):
    workdir=worklist[l]
    pdt = dirt + workdir + "/OVF_grid_T.nc"
    pdu = dirt + workdir + "/OVF_grid_U.nc"
    psave = "%s/%s" % (savedir,workdir)
    try:
        dt = nc4.Dataset(pdt)
        du = nc4.Dataset(pdu)
        print(workdir)
    except :
        print("...failed loading %s" % (pdt))
        continue
    #
    toce    = dt.variables['thetao_inst'][::tskip,:,1,:]
    nT,nK,nI = np.shape(toce)
    tmask = toce[0,:,:] < 1e-10
    bigmask = np.repeat(tmask[np.newaxis,:,:],nT,axis=0)

    toce = np.ma.masked_where(bigmask,toce)
    cfltu   = np.ma.masked_where(bigmask, dt.variables['cfltu'][::tskip,:,1,:])
    cfltw   = np.ma.masked_where(bigmask, dt.variables['cfltw'][::tskip,:,1,:])
    lipst   = np.ma.masked_where(bigmask, dt.variables['lipst'][::tskip,:,1,:])
    dt.close()

    cfluu   = np.ma.masked_where(bigmask, du.variables['cfluu'  ][::tskip,:,1,:]) 
    cfluw   = np.ma.masked_where(bigmask, du.variables['cfluw'  ][::tskip,:,1,:]) 
    lipsu   = np.ma.masked_where(bigmask, du.variables['lipsu'  ][::tskip,:,1,:]) 
    du.close()
    print("data loaded")
    
    optpcflt = {"cmap" : palette, "vmin" : np.min((np.min(cfltw),np.min(cfltu))), 
                                  "vmax" : np.max((np.max(cfltw),np.max(cfltu)))}

    optpcflu = {"cmap" : palette, "vmin" : np.min((np.min(cfluw),np.min(cfluu))), 
                                  "vmax" : np.max((np.max(cfluw),np.max(cfluu)))}

    optplips = {"cmap" : palette, "vmin" : np.min((np.min(lipst),np.min(lipsu))), 
                                  "vmax" : np.max((np.max(lipst),np.max(lipsu)))}


    fig, ax = plt.subplots(NI,NJ, figsize=(NJ*6,NI*4), dpi = dpi, squeeze=False)
    im = [[],[]] ; t=0
    """Set the data for the ith iteration of the animation."""        
    im[0].append( ax[0][0].imshow(toce [t]   , **optptoce)  ) ; ax[0][0].set_title('%s (%d/%d)' % (workdir,t,nT))
    im[0].append( ax[0][1].imshow(cfltw[t]   , **optpcflt)  ) ; ax[0][1].set_title('cfltw max %.3f' % ( np.max(cfltw[t]) ) )
    im[0].append( ax[0][2].imshow(cfluw[t]   , **optpcflu)  ) ; ax[0][2].set_title('cfluw max %.3f' % ( np.max(cfluw[t]) ) )
    im[0].append( ax[0][3].imshow(lipst[t]   , **optplips)  ) ; ax[0][3].set_title('lipst max %.3f' % ( np.max(lipst[t]) ) )
    """ 2nd row : x    cfltu cfluw lipsu
    """
    im[1].append( ax[1][0].axis('off') )
    im[1].append( ax[1][1].imshow(cfltu[t]   , **optpcflt) ) ; ax[1][1].set_title('cfltu max %.3f' % ( np.max(cfltu[t]) ) )
    im[1].append( ax[1][2].imshow(cfluu[t]   , **optpcflu) ) ; ax[1][2].set_title('cfluu max %.3f' % ( np.max(cfluu[t]) ) )
    im[1].append( ax[1][3].imshow(lipsu[t]   , **optplips) ) ; ax[1][3].set_title('lipsu max %.3f' % ( np.max(lipsu[t]) ) )
    #
    def animate(t):
        print("%d/%d" % (t,nT-1))
        """Set the data for the ith iteration of the animation."""      
        im[0][0].set_data(toce [t]) ; ax[0][0].set_title('%s (%d/%d)'     % ( workdir,t,nT     ) )
        im[0][1].set_data(cfltw[t]) ; ax[0][1].set_title('cfltw max %.3f' % ( np.max(cfltw[t]) ) )
        im[0][2].set_data(cfluw[t]) ; ax[0][2].set_title('cfluw max %.3f' % ( np.max(cfluw[t]) ) )
        im[0][3].set_data(lipst[t]) ; ax[0][3].set_title('lipst max %.3f' % ( np.max(lipst[t]) ) )

        """ 2nd row : x    cfltu cfluw lipsu
        """
        im[1][1].set_data(cfltu[t]) ; ax[1][1].set_title('cfltu max %.3f' % ( np.max(cfltu[t]) ) )
        im[1][2].set_data(cfluu[t]) ; ax[1][2].set_title('cfluu max %.3f' % ( np.max(cfluu[t]) ) )
        im[1][3].set_data(lipsu[t]) ; ax[1][3].set_title('lipsu max %.3f' % ( np.max(lipsu[t]) ) )
        #
        return ax
    #
    for i in range(NI):
            for j in range(NJ):
                ax[i][j].set_xticklabels([]) ; ax[i][j].set_yticklabels([])
                ax[i][j].set_xticks([])      ; ax[i][j].set_yticks([]) 
                ax[i][j].set_aspect(aspect='equal')
                ax[i][j].patch.set_color('0.8')
                if (i,j) != (1,0):
                    fig.colorbar(im[i][j],ax=ax[i:(i+1),j:(j+1)], 
                                 shrink=0.8, location='bottom',
                                 pad = 0.02)
    fig.patch.set_facecolor('white')
    anim = animation.FuncAnimation(fig, animate, frames=nT, blit=False, repeat=False)
    writer = animation.writers['ffmpeg'](fps=2)
    anim.save('%s.mp4' % (psave), writer=writer, dpi=dpi)
    plt.close("all")
    print("saved : %s.mp4" % psave)
    #del toce, cfltu, cfltw, lipst, cfluu, cfluw, lipsu
    

moy_phimin0001_smo5
data loaded
0/25
0/25
1/25
2/25
3/25
4/25
5/25
6/25
7/25
8/25
9/25
10/25
11/25
12/25
13/25
14/25
15/25
16/25
17/25
18/25
19/25
20/25
21/25
22/25
23/25
24/25
25/25
saved : up1/moy_phimin0001_smo5.mp4
moy_phimin0001_smo6
data loaded
0/25
0/25
1/25
2/25
3/25
4/25
5/25
6/25
7/25
8/25
9/25
10/25
11/25
12/25
13/25
14/25
15/25
16/25
17/25
18/25
19/25
20/25
21/25
22/25
23/25
24/25
25/25
saved : up1/moy_phimin0001_smo6.mp4
moy_phimin0001_smo7
data loaded
0/25
0/25
1/25
2/25
3/25
4/25
5/25
6/25
7/25
8/25
9/25
10/25
11/25
12/25
13/25
14/25
15/25
16/25
17/25
18/25
19/25
20/25
21/25
22/25
23/25
24/25
25/25
saved : up1/moy_phimin0001_smo7.mp4
moy_phimin01_smo0
data loaded
0/25
0/25
1/25
2/25
3/25
4/25
5/25
6/25
7/25
8/25
9/25
10/25
11/25
12/25
13/25
14/25
15/25
16/25
17/25
18/25
19/25
20/25
21/25
22/25
23/25
24/25
25/25
saved : up1/moy_phimin01_smo0.mp4
moy_phimin01_smo1
data loaded
0/25
0/25
1/25
2/25
3/25
4/25
5/25
6/25
7/25
8/25
9/25
10/25
11/25
12/25
13/25
14/25
15/25
16/25
1